__Cell 1: Install all necessary libraries from requirements.txt__

In [2]:
%pip install -r requirements.txt
%pip install --upgrade gradio

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


__Cell 2: Import all libraries__

In [4]:
import pandas as pd
import gradio as gr
from datetime import datetime, timedelta
import traceback
import warnings

from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import Optional

from entsoe import EntsoePandasClient

__Cell 3: Get ENTSOE API key from entsoe_api_key.txt and get Gemini API key from gemini_api_key.txt__

In [6]:
with open("gemini_api_key.txt", "r") as file:
    for line in file:
        if line.startswith("GEMINI_API_KEY"):
            gemini_api_key = line.split("=")[1].strip()


with open("entsoe_api_key.txt", "r") as file:
    for line in file:       
         if line.startswith("ENTSOE_API_KEY"):
            entsoe_api_key = line.split("=")[1].strip()

__Cell 4: Initialize the Gemini and ENTSOE Clients__

In [8]:
model = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite", google_api_key=gemini_api_key, temperature=0)
client = EntsoePandasClient(api_key=entsoe_api_key)

__Cell 5: Create the class QuestionDetails__

In [10]:
class QuestionDetails(BaseModel):
    function_name: str = Field(description="The exact API function to call based on the user's query.")
    country: str = Field(description="The full English name of the country.")
    start_date: str = Field(description="The start date for the query in 'YYYY-MM-DD' format.")
    end_date: str = Field(description="The end date for the query in 'YYYY-MM-DD' format.")

__Cell 6: Set up Pydantic Parser__

In [12]:
parser = PydanticOutputParser(pydantic_object=QuestionDetails)

__Cell 7: Create the prompt__

In [14]:
prompt_string = """
You are an expert API assistant for the ENTSO-E energy platform. Your job is to parse a user's query and convert it into a structured JSON object.

Today's date is {current_date}.

--- FUNCTION MAPPING ---
Use this table to determine the correct 'function_name'.

| If the user asks for...             | Use this 'function_name'      |
|-------------------------------------|-------------------------------|
| Actual load, consumption, or demand | query_load                    |
| Generation forecast or prediction   | query_generation_forecast     |

--- PARAMETER EXTRACTION RULES ---
1.  **Country**: Extract the full English country name from the user's query.
2.  **Date Range**:
    - Based on the query and today's date ({current_date}), figure out the correct `start_date` and `end_date` in 'YYYY-MM-DD' format.
    - If the user says "yesterday", use yesterday's date.
    - If no date is mentioned, assume the user means "today".

--- EXAMPLES ---
- Query: "What is the current load in France?"
  Expected Output: {{ "function_name": "query_load", "country": "France", "start_date": "2025-09-08", "end_date": "2025-09-08" }}

- Query: "Show me the generation forecast for Germany today"
  Expected Output: {{ "function_name": "query_generation_forecast", "country": "Germany", "start_date": "2025-09-08", "end_date": "2025-09-08" }}

- Query: "What was the load in Spain yesterday?"
  Expected Output: {{ "function_name": "query_load", "country": "Spain", "start_date": "2025-09-07", "end_date": "2025-09-07" }}

{format_instructions}

User Query:
{query}
"""

__Cell 8: Create the prompt template__

In [16]:
prompt = PromptTemplate(
    template=prompt_string,
    input_variables=["query", "current_date"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

__Cell 9: Create langchain chain__

In [18]:
chain = prompt | model | parser

__Cell 10: List of Country Codes__

In [20]:
EUROPEAN_COUNTRY_CODES = {
    "albania": "AL", "austria": "AT", "belgium": "BE", "bosnia and herzegovina": "BA",
    "bulgaria": "BG", "croatia": "HR", "cyprus": "CY", "czech republic": "CZ", "denmark": "DK",
    "estonia": "EE", "finland": "FI", "france": "FR", "georgia": "GE", "germany": "DE",
    "greece": "GR", "hungary": "HU", "iceland": "IS", "ireland": "IE", "italy": "IT",
    "kosovo": "XK", "latvia": "LV", "lithuania": "LT", "luxembourg": "LU", "malta": "MT",
    "moldova": "MD", "montenegro": "ME", "netherlands": "NL", "north macedonia": "MK", 
    "norway": "NO", "poland": "PL", "portugal": "PT", "romania": "RO", "serbia": "RS",
    "slovakia": "SK", "slovenia": "SI", "spain": "ES", "sweden": "SE", "switzerland": "CH",
    "turkey": "TR", "ukraine": "UA", "united kingdom": "GB",
}


__Cell 11: Create function that retrieves the data using the ENTSOE API__

In [22]:
def call_entsoe_api_dispatcher(client, question: QuestionDetails):
    start = pd.Timestamp(question.start_date, tz='Europe/Brussels')
    end = pd.Timestamp(question.end_date, tz='Europe/Brussels') + timedelta(days=1)
    
    country_code = EUROPEAN_COUNTRY_CODES.get(question.country.lower()) if question.country else None

    if not country_code:
        return "Query is missing a valid country."

    func_name = question.function_name
    
    try:
        if func_name == 'query_load':
            return client.query_load(country_code, start=start, end=end)
            
        elif func_name == 'query_generation_forecast':
            return client.query_generation_forecast(country_code, start=start, end=end)
            
        else:
            return f"Function '{func_name}' is not supported by this dispatcher."

    except Exception as e:
        return f"API call failed for function '{func_name}' with error: {e}"

__Cell 12: Formats the data into a summary string and a pandas DataFrame for table display__

In [24]:
def format_data_as_table(data, details: QuestionDetails):
    title = details.function_name.replace('query_', '').replace('_', ' ').title()
    
    if isinstance(data, str):
        return data, None
    
    if data.empty:
        return f"No data found for {title} in {details.country.capitalize()}", None

    try:
        stats_series = data.iloc[:, 0] if isinstance(data, pd.DataFrame) else data
        
        summary = (
            f"**Summary for {title} in {details.country.capitalize()}**\n"
            f"**Period:** {details.start_date} to {details.end_date}\n\n"
            f"Mean: **{stats_series.mean():.2f}**\n"
            f"Min: **{stats_series.min():.2f}**\n"
            f"Max: **{stats_series.max():.2f}**"
        )
    except Exception as e:
        summary = f"**Data for {title} in {details.country.capitalize()}**\n(Could not calculate summary statistics due to an error.)"

    unit = "EUR/MWh" if 'price' in details.function_name.lower() else "MW"
    table_df = pd.DataFrame(data).reset_index()
    
    if len(table_df.columns) >= 2:
        table_df.columns = ['Timestamp'] + [f'Value_{i}' for i in range(1, len(table_df.columns))]
        table_df.rename(columns={'Value_1': f'Value ({unit})'}, inplace=True)
    
    table_df['Timestamp'] = pd.to_datetime(table_df['Timestamp']).dt.strftime('%Y-%m-%d %H:%M')
    
    return summary, table_df

__Cell 13: Create the main function__

In [26]:
def process_query(question):
    if not question:
        return "Please ask a question.", None
    try:
        current_date_str = datetime.now().strftime('%Y-%m-%d')
        parsed_details = chain.invoke({"query": question, "current_date": current_date_str})
        
        data = call_entsoe_api_dispatcher(client, parsed_details)
        
        summary, table = format_data_as_table(data, parsed_details)
        return summary, table
    except Exception as e:
        traceback.print_exc()
        return f"Sorry, a critical error occurred. Error: {e}", None

__Cell 14: Create the Gradio Interface__

In [28]:
interface = gr.Interface(
    fn=process_query,
    inputs=gr.Textbox(lines=2, placeholder="e.g., Show me electricity prices in Germany yesterday..."),
    outputs=[
        gr.Markdown(label="Summary"),
        # Replace 'max_rows=15' with 'height=500' for visual control
        gr.DataFrame(label="Data Table", wrap=True) 
    ],
    title="🇪🇺 European Energy Data Chatbot (Table View)",
    description="Ask me about day-ahead electricity prices or total load for various European countries.",
    flagging_mode="never"
)

__Cell 15: Launch the interface and use the chatbot and use the library warnings to remove some unnecessary error messages that are not relevant__

In [30]:
interface.launch()
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=ResourceWarning, message="unclosed <socket.socket.*>")

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


/opt/anaconda3/lib/python3.12/site-packages/uvicorn/protocols/websockets/websockets_impl.py:17: DeprecationWarning: websockets.server.WebSocketServerProtocol is deprecated
  from websockets.server import WebSocketServerProtocol
